In [ ]:
%matplotlib inline
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 150, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(150, 50, 5)
        self.fc1 = nn.Linear(50 * 5 * 5, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50,10)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
  
        return x


net = Net()


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)



Train_acc = []
Test_acc = []

def accuracy(X):
  correct = 0
  total = 0
  with torch.no_grad():
      for data in X:
          images, labels = data
          outputs = net(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  return correct * 100 // total

for epoch in range(30):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1250 == 1249:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 1250:.3f}')
            running_loss = 0.0
    train_accuracy = accuracy(trainloader)
    test_accuracy = accuracy(testloader)
    print("Test Accuracy = ",test_accuracy)
    print("Train_accuracy = ",train_accuracy)
    Train_acc.append(train_accuracy)
    Test_acc.append(test_accuracy)

print('Finished Training')

Files already downloaded and verified
Files already downloaded and verified
[1,  1250] loss: 2.138
[1,  2500] loss: 1.815
[1,  3750] loss: 1.665
[1,  5000] loss: 1.547
[1,  6250] loss: 1.500
[1,  7500] loss: 1.415
[1,  8750] loss: 1.397
[1, 10000] loss: 1.319
[1, 11250] loss: 1.310
[1, 12500] loss: 1.247
Test Accuracy =  57
Train_accuracy =  58
[2,  1250] loss: 1.172
[2,  2500] loss: 1.170
[2,  3750] loss: 1.141
[2,  5000] loss: 1.121
[2,  6250] loss: 1.103
[2,  7500] loss: 1.080
[2,  8750] loss: 1.033
[2, 10000] loss: 1.053
[2, 11250] loss: 1.048
[2, 12500] loss: 1.028
Test Accuracy =  64
Train_accuracy =  67
[3,  1250] loss: 0.935
[3,  2500] loss: 0.929
[3,  3750] loss: 0.954
[3,  5000] loss: 0.926
[3,  6250] loss: 0.919
[3,  7500] loss: 0.897
[3,  8750] loss: 0.880
[3, 10000] loss: 0.899
[3, 11250] loss: 0.894
[3, 12500] loss: 0.876
Test Accuracy =  67
Train_accuracy =  72
[4,  1250] loss: 0.770
[4,  2500] loss: 0.775
[4,  3750] loss: 0.794
[4,  5000] loss: 0.811
[4,  6250] loss: 0.

KeyboardInterrupt: ignored

In [ ]:
print("The values of hyper parameters are:")
print("Learning Rate=",0.001)
print("Momentum=",0.9)
print("M=",150)
print("Max Train and Test accuracies are:")
print(max(Train_acc))
print(max(Test_acc))

The values of hyper parameters are:
Learning Rate= 0.001
Momentum= 0.9
M= 150
Max Train and Test accuracies are:
93
72
